<a href="https://colab.research.google.com/github/nalet/bme.Advanced-Topics-in-Machine-Learning/blob/master/Assignments/Assignment2/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Name: Nalet Meinen¶

# Assignment 2 ATML 2020
## Classification with limited data
ImageNet is a well known dataset with 1000 image classes. We will be working on a subset of the dataset (60k images, 100 classes, 600 images per class 80$\times$80 pixels, RGB) and train a model to classify an image into one of the 100 classes. The dataset is located under the "data" directory. Training and validation data splits are under "data/train" and "data/val" directories respectively. Both splits consist of 100 directories, each representing an object category.

In [1]:
import os.path
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)
if not os.path.isfile('data.zip'):
  from google.colab import drive
  drive.mount('/content/drive')
  !cp -r "/content/drive/My Drive/ATML/Assignments/Assignment2/data.zip" "data.zip"
  !unzip -qq data.zip

Fri Apr 17 15:21:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Task 1. Implement ImageNetLimited class for data loading in datasets.py file

In [0]:
from PIL import Image
import glob
import torch
import os.path

class ImageNetLimited(torch.utils.data.Dataset):
    """ImageNet Limited dataset."""
    
    def __init__(self, root_dir, transform, instances=1):
        data = []

        for path in tqdm(glob.glob(root_dir + "/**/*")):
          target = int(os.path.basename(os.path.dirname(path)))
          _image = Image.open(path)
          
          for _ in range(instances):
            data.append((transform(_image),target))
        
        self.n = len(data)
        self.data = data
        #   if(augument):
        #     data.append((sample[0], sample[1])) #default
        #     data.append((sample[0].transpose(1,2).flip(2), sample[1])) #default 90
        #     #data.append((sample[0].flip(1), sample[1])) # default 180
        #     #data.append((sample[0].transpose(1,2), sample[1])) # default 270

        #     data.append((sample[0].flip(2), sample[1])) #flip 
        #     #data.append((sample[0].flip(2).transpose(1,2).flip(2), sample[1])) #flip 90
        #     #data.append((sample[0].flip(2).flip(1), sample[1])) #flip 180
        #     data.append((sample[0].flip(2).transpose(1,2), sample[1])) #flip 270
        #   else:
        #     data.append(sample)
        # self.n = len(root_dir)
        # self.data = data

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        return self.data[idx]
        

In [3]:
import numpy as np
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print("This nootebook ran on",device.type,"With number of GPU:",torch.cuda.device_count())

train_dir = 'data/train'
validation_dir = 'data/val'

# write your code
_transforms_train = transforms.Compose([
    transforms.RandomCrop(64),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation([-90,90]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

_transforms_val = transforms.Compose([
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# _train_dataset = ImageFolder(train_dir, transform=_transforms)
# _val_dataset = ImageFolder(validation_dir, transform=_transforms)

train_dataset = ImageNetLimited(train_dir,_transforms_train,4)
val_dataset = ImageNetLimited(validation_dir,_transforms_val)
print("Train dataset length:",len(train_dataset),"Validation dataset length:",len(val_dataset))

batch_size = 128
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8) #set num_workers = 0, to fix BrokenPipeError in Windows
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

This nootebook ran on cuda With number of GPU: 1



Train dataset length: 240000 Validation dataset length: 29855


## Task 2. CNN Architecture
Design and implement a Convolutional Neural Network architecture for image classification in a **ConvNet** class in the notebook. Some examples of popular classification models are: AlexNet, VGG, ResNet, ... Justify your design choices in the report. The input to your model must be an image of size $64 \times 64$ pixels.

In [0]:
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

class BasicBlock(nn.Module):
  expansion = 1

  def __init__(self, in_planes, planes, stride=1):
    super(BasicBlock, self).__init__()
    self.conv1 = nn.Conv2d(
        in_planes, planes, kernel_size=5, stride=stride, padding=2, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(
        planes, planes, kernel_size=5, stride=1, padding=2, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != self.expansion * planes:
      self.shortcut = nn.Sequential(
          nn.Conv2d(
              in_planes,
              self.expansion * planes,
              kernel_size=1,
              stride=stride,
              bias=False), nn.BatchNorm2d(self.expansion * planes))

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    out += self.shortcut(x)
    out = F.relu(out)
    return out


class ResNet(nn.Module):

  def __init__(self, block, num_blocks, num_classes=100):
    super(ResNet, self).__init__()
    self.in_planes = 64

    self.conv1 = nn.Conv2d(
        3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self._make_layer(block, 384, num_blocks[3], stride=2)
    #self.linear = nn.Linear(512 * block.expansion, num_classes)
    self.linear = nn.Linear(384 * block.expansion * 4, num_classes)

  def _make_layer(self, block, planes, num_blocks, stride):
    strides = [stride] + [1] * (num_blocks - 1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_planes, planes, stride))
      self.in_planes = planes * block.expansion
    return nn.Sequential(*layers)

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = F.avg_pool2d(out, 4)
    out = torch.flatten(out, 1)
    out = self.linear(out)
    return F.log_softmax(out, dim=1)


def ConvNet():
  return ResNet(BasicBlock, [1, 1, 1, 1])

# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         self.features = nn.Sequential(
#             nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=0),

#             nn.BatchNorm2d(64),
#             nn.ReLU(inplace=True),

#             nn.MaxPool2d(kernel_size=2, stride=2),

#             nn.Conv2d(64, 256, kernel_size=5, padding=1),

#             nn.BatchNorm2d(256),
#             nn.ReLU(inplace=True),

#             nn.MaxPool2d(kernel_size=2, stride=2),

#             nn.Conv2d(256, 384, kernel_size=3, padding=0),

#             nn.BatchNorm2d(384),
#             nn.ReLU(inplace=True),

#             nn.Conv2d(384, 192, kernel_size=3, padding=1),

#             nn.BatchNorm2d(192),
#             nn.ReLU(inplace=True),

#             nn.MaxPool2d(kernel_size=6, stride=6),
#         )
#         self.classifier = nn.Sequential(
#             nn.Dropout(0.5),
#             nn.Linear(192 * 2 * 2, 1024),
#             nn.BatchNorm1d(1024),
#             nn.ReLU(inplace=True),
#             nn.Linear(1024, 100),
#         )
        
#     def forward(self, out):
#         out = self.features(out)
#         out = out.view(out.size(0), 192 * 2 * 2)
#         out = self.classifier(out)
#         return out

# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

## Task 3. Train Model
Implement training and evaluation code for your model. Choose an appropriate loss function and evaluate the model on the validation set using classification accuracy. You are not allowed to use a pre-trained model (must train from scratch on the provided data).<br>
<font color='red'>Your model should achieve an accuracy of at least 40.0% on the validation set (Model with performance smaller than 40.0% will result in 0 points for this task).</font><br>

In [0]:
import numpy as np
import torch
from tqdm.notebook import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def train(model, train_loader, optimizer, loss_fn, print_every=100):
    '''
    Trains the model for one epoch
    '''
    model.train()
    losses = []
    n_correct = 0
    for iteration, (images, labels) in enumerate(tqdm(train_loader)):
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        optimizer.zero_grad()
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
#        if iteration % print_every == 0:
#            print('Training iteration {}: loss {:.4f}'.format(iteration, loss.item()))
        losses.append(loss.item())
        n_correct += torch.sum(output.argmax(1) == labels).item()
    accuracy = 100.0 * n_correct / len(train_loader.dataset)
    return np.mean(np.array(losses)), accuracy
            
def test(model, test_loader, loss_fn):
    '''
    Tests the model on data from test_loader
    '''
    model.eval()
    test_loss = 0
    n_correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            loss = loss_fn(output, labels)
            test_loss += loss.item()
            n_correct += torch.sum(output.argmax(1) == labels).item()

    average_loss = test_loss / len(test_loader)
    accuracy = 100.0 * n_correct / len(test_loader.dataset)
    print('Test average loss: {:.4f}, accuracy: {:.3f}'.format(average_loss, accuracy))
    return average_loss, accuracy


def fit(train_dataloader, val_dataloader, model, optimizer, loss_fn, n_epochs, scheduler=None):
    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []

    for epoch in tqdm(range(n_epochs)):
        train_loss, train_accuracy = train(model, train_dataloader, optimizer, loss_fn)
        val_loss, val_accuracy = test(model, val_dataloader, loss_fn)
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        if scheduler:
            scheduler.step() # argument only needed for ReduceLROnPlateau
        print('Epoch {}/{}: train_loss: {:.4f}, train_accuracy: {:.4f}, val_loss: {:.4f}, val_accuracy: {:.4f}'.format(epoch+1, n_epochs,
                                                                                                          train_losses[-1],
                                                                                                          train_accuracies[-1],
                                                                                                          val_losses[-1],
                                                                                                          val_accuracies[-1]))
    
    return train_losses, train_accuracies, val_losses, val_accuracies

In [0]:
model = ConvNet() #resnet18
model = model.to(device)
learning_rate = 0.02
optimizer = torch.optim.SGD(model.parameters(), learning_rate, momentum=0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
#optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
n_epochs = 300
loss_fn = nn.CrossEntropyLoss()
print(model)
curves_conv2 = fit(train_dataloader, val_dataloader, model, optimizer, loss_fn, n_epochs)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      (bn2): BatchNorm2d(128,


Test average loss: 3.3488, accuracy: 18.556
Epoch 1/300: train_loss: 3.6168, train_accuracy: 14.0567, val_loss: 3.3488, val_accuracy: 18.5564



Test average loss: 2.8910, accuracy: 26.639
Epoch 2/300: train_loss: 2.9539, train_accuracy: 25.4229, val_loss: 2.8910, val_accuracy: 26.6388


## Task 4. Ablations
Try to find the best performing model by tuning the model design and hyper-parameters on the validation set. Perform ablation experiments to illustrate the effect of the most important hyper-parameters. Some examples of ablations: training parameters (e.g., optimizer, learning rates, batch size), network architecture (e.g., number of layers, number of units, activation function, normalization layers), model regularization (e.g., data augmentation, dropout, weight decay, early stopping), test-time augmentation, etc...  <br>**Perform at least 5 ablations and report the performance of each on the validation set.**

In [0]:
# write your code

## Task 5. Model Errors
Evaluate the trained model on the validation set and plot 10 random mistakes that your model made.

In [0]:
# write your code

## Task 6. Competition time!
Read the images from "data/test" folder. There are no labels for these images. Run your best model on these images and save the image IDs (names) and predicted label in a file LastName.csv. You will receive a link via email to upload the CSV file to  an online system which will give you the score of your model on the held-out test set. Top 5 students with at least 40% classification accuracy will obtain bonus points.

In [0]:
# write your code